# Modelling patch-clamp experiments: simplified models

In the [previous notebook](./artefacts-2-compensation.ipynb) we presented an update model of patch-clamp experiments that included electronic compensation.
Here, we show how this model can be simplified by omitting the fastest processes.

We start from the main schematic and equations.

<img src="resources/patch-comp-3-Rs.png" style="margin:auto" />

_**Figure 1**: The full patch-clamp schematic_

\begin{align}
C1. && C_m\dot{V}_m = \frac{V_p + E_\text{off}^\dagger - V_m}{R_s} - \frac{V_m - E_\text{leak}}{R_\text{leak}} - I
\end{align}

\begin{align}
C2. && (C_p+C_f)\dot{V}_p = \frac{V_o - V_p}{R_f} + C_f\dot{V}_o  - \frac{V_p + E_\text{off}^\dagger - V_m}{R_s} + (C_p^* + C_m^*) \dot{V}_\text{ref}
\end{align}

\begin{align}
C3. && \tau_a \dot{V}_o = V_\text{ref} - V_p
\end{align}

\begin{align}
C4. && \tau_\text{sum} \dot{V}_\text{ref} = V_c + \alpha R_s^*I_\text{obs} - V_\text{ref}
\end{align}

\begin{align}
C5. && R_f I_\text{obs} = V_o - V_\text{ref}
\end{align}

## Conclusion